In [1]:
import sys
sys.path.insert(0,'/home/gustavu92/Desktop/unicamp/tools/')

In [2]:
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from train import SegDataset
from model.build_BiSeNet import BiSeNet
import albumentations as A
import evaluation
from data_loader import SegDataset

In [3]:
model = BiSeNet(3, 'resnet18').cuda()

In [4]:
model.load_state_dict(torch.load('./checkpoints/test1/best_model.pth'))
model.eval()

BiSeNet(
  (saptial_path): Spatial_path(
    (convblock1): ConvBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock3): ConvBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (context_path): resnet18(
    (features): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [5]:
X_train = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_train.pkl')
X_val = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_val.pkl')
X_test = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_test.pkl')
y_train = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_train.pkl')
y_val = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_val.pkl')
y_test = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_test.pkl')


shape = (384, 512)
batch_size = 16
num_workers = 4

val_dataset = SegDataset(X_val, y_val, shape, None)
test_dataset = SegDataset(X_test, y_test, shape, None)

dataloader_val = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

dataloader_test = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

In [6]:
def metrics(y_true, y_pred, channel=None):
    pred = torch.argmax(y_pred, 1)
    pred = (pred==channel).byte()
    pred_inv = (~pred)
    gt = (y_true == channel).byte()
    gt_inv = (~gt)
    tp = (pred & gt).sum()
    tn = (pred_inv & gt_inv).sum()
    fp = (pred & gt_inv).sum()
    fn = (pred_inv & gt).sum()
    accuracy  = (tp + tn)/(tp + tn + fp + fn).float()
    precision  = tp/(tp + fp).float()
    recall = tp/(tp + fn).float()
    return torch.Tensor([accuracy, precision, recall])

In [7]:
metrics_accumulated_soil = torch.zeros((3)).float()
metrics_accumulated_crop = torch.zeros((3)).float()
metrics_accumulated_weed = torch.zeros((3)).float()
start = time.time()

with torch.no_grad():
    for i, (x_val, y_val) in enumerate(dataloader_val):
        x_val = x_val.cuda(non_blocking=True)
        y_val = y_val.cuda(non_blocking=True)

        y_val_pred = model(x_val)
        metrics_accumulated_soil += metrics(y_val, y_val_pred, 0)
        metrics_accumulated_crop += metrics(y_val, y_val_pred, 1)
        metrics_accumulated_weed += metrics(y_val, y_val_pred, 2)
        
#         for j in range(len(x_val)):
#             f, ax = plt.subplots(1,3)
#             f.set_figheight(15)
#             f.set_figwidth(15)
#             ax[0].imshow(x_val.permute(0, 3, 2, 1).cpu().numpy()[j])
#             ax[1].imshow(y_val.permute(0, 2, 1).cpu().numpy()[j])
#             ax[2].imshow(np.argmax(y_val_pred.permute(0, 3, 2, 1).cpu().numpy(), -1)[j])
#             plt.show()
        
#         if i == 50:
#             break

end = time.time()
metrics_mean_soil = metrics_accumulated_soil/len(dataloader_val)
metrics_mean_crop = metrics_accumulated_crop/len(dataloader_val)
metrics_mean_weed = metrics_accumulated_weed/len(dataloader_val)

fps = len(dataloader_val.dataset)/(end - start)
print(metrics_mean_crop)
print(metrics_mean_weed)
print(fps)

tensor([0.9970, 0.8912, 0.9127])
tensor([0.9980, 0.6640, 0.5146])
79.93765304735508


In [8]:
metrics_accumulated_soil = torch.zeros((3)).float()
metrics_accumulated_crop = torch.zeros((3)).float()
metrics_accumulated_weed = torch.zeros((3)).float()
start = time.time()

with torch.no_grad():
    for i, (x_test, y_test) in enumerate(dataloader_test):
        x_test = x_test.cuda(non_blocking=True)
        y_test = y_test.cuda(non_blocking=True)

        y_test_pred = model(x_test)
        metrics_accumulated_soil += metrics(y_test, y_test_pred, 0)
        metrics_accumulated_crop += metrics(y_test, y_test_pred, 1)
        metrics_accumulated_weed += metrics(y_test, y_test_pred, 2)
        
#         for j in range(len(x_val)):
#             f, ax = plt.subplots(1,3)
#             f.set_figheight(15)
#             f.set_figwidth(15)
#             ax[0].imshow(x_val.permute(0, 3, 2, 1).cpu().numpy()[j])
#             ax[1].imshow(y_val.permute(0, 2, 1).cpu().numpy()[j])
#             ax[2].imshow(np.argmax(y_val_pred.permute(0, 3, 2, 1).cpu().numpy(), -1)[j])
#             plt.show()
        
#         if i == 50:
#             break

end = time.time()
metrics_mean_soil = metrics_accumulated_soil/len(dataloader_test)
metrics_mean_crop = metrics_accumulated_crop/len(dataloader_test)
metrics_mean_weed = metrics_accumulated_weed/len(dataloader_test)

fps = len(dataloader_test.dataset)/(end - start)
print(metrics_mean_crop)
print(metrics_mean_weed)
print(fps)

tensor([0.9969, 0.8912, 0.9140])
tensor([0.9979, 0.6607, 0.5222])
79.66069508076517
